In [1]:
from coc131_cw import *

#### **Question 1 Check**

In [ ]:
coc = COC131()
res1, res2 = coc.q1('AnnualCrop_1.jpg')
print(f"res1: {res1}") 
print(f"res2: {res2}")
print(f"res1 shape: {res1.shape}")


res1: [149. 121. 120. ... 106.  94. 104.]
res2: AnnualCrop
res1 shape: (3072,)


#### **Question 2 Check**

#### **Question 3 Check**

#### **Question 4 Check**

#### **Question 5 Check**

#### **Question 6 Check**